In [ ]:
pip install pyspark

In [ ]:
import pyspark
from pyspark import SparkContext
sc = SparkContext("local", "BDA")

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import json
json_file = sc.textFile("Software_5.json")
records = json_file.map(lambda js: json.loads(js)).filter(lambda dic: 'reviewerID' in dic and 'asin' in dic and 'overall' in dic and 'unixReviewTime' in dic)
key_records = records.map(lambda record: (record['reviewerID'], record['asin'], record['overall']))
reviewerID_key_records = key_records.map(lambda t: (t[0], (t[1], t[2])))

# key_records.take(1000)
# user1_recs = key_records.filter(lambda record: record[0] == 'A38NELQT98S4H8')
# group_by_user1 = user1_recs.groupByKey().map(lambda t: (t[0], list(t[1])))
# group_by_user1.take(20)

def latest_rating(asin_rating_kvs):
   res = []
   asin_rating_dict = dict()
   for (asin,rating) in asin_rating_kvs:
      try:
         asin_rating_dict[asin].append(rating)
      except KeyError:
         asin_rating_dict[asin] = [rating]
   for (asin, ratings) in asin_rating_dict.items():
      res.append((asin, ratings[-1]))
   return res


grouped_by_reviewer = reviewerID_key_records.groupByKey()
grouped_by_asin = asin_key_records.groupByKey()

filtered_user_latest_rating = grouped_by_reviewer.map(lambda t: (t[0], latest_rating(list(t[1]))))
flat_records_reviewer_key = filtered_user_latest_rating.flatMapValues(lambda t: t)
flat_records_asin_key = flat_records_reviewer_key.map(lambda t: (t[1][0], (t[0], t[1][1])))
grouped_by_asin = flat_records_asin_key.groupByKey().map(lambda t: (t[0], list(t[1])))

grouped_by_asin.take(10)




[('0321719816',
  [('A38NELQT98S4H8', 4.0),
   ('A3QJU4FEN8PQSZ', 4.0),
   ('ACJT8MUC0LRF0', 5.0),
   ('AYUF7YETYOLNX', 5.0),
   ('A31ICLWQ9CSHRS', 5.0),
   ('A2BVNVJOFXGZUB', 5.0),
   ('A2JMJVNTBL7K7E', 5.0),
   ('A14JBDSWKPKTZA', 5.0),
   ('A2WCFDOCS73PNB', 5.0),
   ('A14638TGYH7GD9', 5.0),
   ('A2VWE5SGNDS8HW', 3.0),
   ('A3M1PA1C98XNGN', 4.0),
   ('AMRMK86X3PKXD', 5.0),
   ('A2FRAS05LEI6JJ', 4.0)]),
 ('B000XHE0Q0',
  [('A38NELQT98S4H8', 4.0),
   ('A1GQRGB8FGSLIZ', 3.0),
   ('A1GWG5CWLKJ7ET', 4.0),
   ('A3CAG7WLTAKPI', 5.0),
   ('A7R79JLQ5Q28M', 4.0),
   ('A1E0XB78OUMY2T', 1.0),
   ('A1F1A0QQP2XVH5', 3.0),
   ('A21S26XYPGXJZX', 4.0),
   ('A31HE83MU98GBV', 4.0),
   ('AG1OL1K3SFURN', 3.0),
   ('A3Q1J68QY1MZQ9', 4.0),
   ('A22TZUXUKA11SQ', 2.0),
   ('A13JDHPNAD2L0P', 1.0),
   ('A3RGHO0Q938FRS', 5.0),
   ('A3KGLXW3EYDTUH', 4.0),
   ('A119S8FMR47N2N', 4.0),
   ('A14NA0W8ESGDSI', 4.0),
   ('A3KZCNV1N8R9GC', 4.0),
   ('ASQK4UT56QBTI', 4.0),
   ('A1LH6RF4UN9VI6', 3.0),
   ('A12E0Y0J6584RT',